In [85]:
import os

print([soundtrack for soundtrack in os.listdir('./') if soundtrack.endswith(".wav")])

['002_sample.wav', '001_sample.wav']


In [5]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english")

10/10/2023 15:38:41 - INFO - huggingsound.speech_recognition.model - Loading model...


In [86]:
audio_paths = [soundtrack for soundtrack in os.listdir('./') if soundtrack.endswith(".wav")]
audio_paths

['002_sample.wav', '001_sample.wav']

In [10]:
transcriptions = model.transcribe(audio_paths)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.82s/it]


In [12]:
print(transcriptions[0]['transcription'])

my name is suno and like pixzabut i also have other interest such as plantitacto and


In [87]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

In [88]:
import librosa
from datasets import Dataset

sr_default = 16_000

def speech_file_to_array_fn(audio_path):
    speech_array, sampling_rate = librosa.load(audio_path, sr=sr_default)
    return speech_array

result = list(map(speech_file_to_array_fn, audio_paths))
inputs = processor(result, sampling_rate=sr_default, return_tensors="pt", padding=True)

In [89]:
import torch

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

In [90]:
predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)

In [91]:
for i, predicted_sentence in enumerate(predicted_sentences):
    print("Reference:", audio_paths[i])
    print("Prediction:", predicted_sentence)

Reference: 002_sample.wav
Prediction: hallo my dogter is cut
Reference: 001_sample.wav
Prediction: my name is suno and like pixzabut i also have other interest such as plantitacto and
